<font size="+3" ><b> <center>Learning XGBoost using Kaggle House Price competition</center></b></font>
>
_**Objectives**_
>
> This is a work in progress, not a polished, complete notebook
>
- Explore Kaggle House Price dataset
- Learn XGBoost
- Learn XGBoost inbuilt k-Fold crossvalidation
- Learn GridsearchCV
- Learn RandomSerchCV
- Learn hyperopt
- Compare model accuracy scores with Kaggle competition submission scores

This is a _long_ Notebook, so care has been taken to use a hierarchy for sections.  Suggestions for how to make this Notebook more user friendly will be much appreciate.
>
_With acknowledgement to [Machine Learning A-Z™: Hands-On Python & R In Data Science](https://www.udemy.com/course/machinelearning/)_
>

# Importing the libraries

In [62]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import qgrid  #   To explore Pandas DataFrames like a "spreasheet"
from sklearn.preprocessing import OneHotEncoder
import math
from IPython.core.interactiveshell import InteractiveShell

#  For measuring model performance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#  Make sure *all* print() lines are printed, not just the last one
InteractiveShell.ast_node_interactivity = "all"

# Make sure matplotlib charts and graphs are displayed in the cell outputs
%matplotlib inline

# Import and explore the dataset

In [63]:
#   This template assumes the data file contains the dependent (y) variable
#   in the *last column*

#   Load training data
training_data = pd.read_csv('hp_train.csv')  #  Kaggle house price training data
X = training_data.iloc[:, :
                       -1]  #  To get Numpy ndarray add .value.  This gives Pandas DataFrame
y = training_data.iloc[:,
                       -1]  #  To get Numpy ndarray add .value.  This gives Pandas DataFrame

# NB: convert y into type float
y = y.astype(float)  #  This makes y a Pandas Series

#   Load test data for competition
competition_data = pd.read_csv('hp_test.csv')  #  Kaggle house price test data

>*Note*
>
>Categorical features not supported
>
>Note that XGBoost does not provide specialization for categorical features; if your data contains categorical features, load it as a NumPy array first and then perform corresponding preprocessing steps like one-hot encoding.

# Processing the data
>
> With acknowledgement to [Kaggle Notebook by Dominik Gawlik](https://www.kaggle.com/dgawlik/house-prices-eda) for a lot of the techniques used

## Dealing with missing data

XGBoost can automatically handle missing data.  XGBoost was designed to work with sparse data.  However, using k-Nearest Neighbour can, under certain circumstances, beat the built-in function.  _(Acknowledgement to [Massimo Belloni](https://towardsdatascience.com/xgboost-is-not-black-magic-56ca013144b4))_.  This is generally where there is a high number of missing values.  In this Notebook the kNN method is followed for learning purposes.

To outperform the XGBoost built-in default strategy we need two things:

* A distance metric that takes into account missing values [(see this post by AirBnb)](https://medium.com/airbnb-engineering/overcoming-missing-values-in-a-random-forest-classifier-7b1fc1fc03ba)
* To normalise the dataset to have meaningful distances, obtained by summing up differences among features with different domains (this is not strictly required by XGBoost but it’s needed for kNN imputation)  _**Note:**_ This normalisation should not hurt XGBoost performance - see answer by [Sycorax](https://stats.stackexchange.com/users/22311/sycorax) on [StackExchange](https://stats.stackexchange.com/questions/353462/what-are-the-implications-of-scaling-the-features-to-xgboost)

For resources on using kNN to impute missing values, see posts by:

* [Yohan Obadia](https://towardsdatascience.com/the-use-of-knn-for-missing-values-cf33d935c637)
* [Kaushik](https://www.analyticsvidhya.com/blog/2020/07/knnimputer-a-robust-way-to-impute-missing-values-using-scikit-learn/)
* [Jason Brownlee](https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/)

### Explore missing data

Using the **missingno** library as explained by [Soner Yildirim](https://towardsdatascience.com/visualize-missing-values-with-missingno-ad4d938b00a1)

In [ ]:
# Import the missingno library
import missingno as msno

In [ ]:
# Make DataFrame with only missing data
# Note the full training data set is used, which includes y - the dependent variable

# isna returns missing values
missingdata = (training_data.isna().sum() > 0
              )  # this is a series with dtype: bool
missingdata = missingdata.to_numpy(
)  # converts series into numpy array, so now have a boolean array
only_missing_data = training_data[training_data.columns[
    missingdata]]  # data.columns returns an index, convert this to a DataFrame
only_missing_data  # this is now a DataFrame containing only the columns and rows where there is missing data in the column

In [ ]:
# Use qgrid to show lika a spreadsheet - https://github.com/quantopian/qgrid
import qgrid
qgrid_widget = qgrid.show_grid(only_missing_data, show_toolbar=True)
qgrid_widget

In [ ]:
# Want to sort the dataframe columns based on number of missing values in each column

number_missing_data = only_missing_data.isna().sum().to_numpy(
)  # array with number of missing values in each column
column_names = only_missing_data.columns.to_numpy(
)  # this gives an array containing column names
missing = np.vstack(
    (column_names, number_missing_data
    ))  # make array by combining number of missing data and column name
missing = pd.DataFrame(
    missing
)  # convert to pandas dataframe with row 0 containing column names, row 1 sum of missing data
missing = missing.T  # transpose - first column now column names, second column sum of missing values
missing = missing.sort_values(
    by=1, axis=0, ascending=False
)  # column names = 0,1, so sort on second column, largest to smallest
missing = missing.T  # missing now has sorted column names in first row
missing


>Conclusion: the majority of missing data is in PoolQC, MiscFeature, Alley, Fence and FireplaceQu


In [ ]:
# now sort DataFrame "only_missing_data" columns based on column order in "missing" DataFrame
column_order = missing.iloc[0, :]  # this is a series
column_order = column_order.to_numpy()  # convert series to array
#column_order
only_missing_data = only_missing_data.reindex(column_order, axis=1)
only_missing_data

In [ ]:
### Visualize missing data with missingno

#### Matrix plot

In [ ]:
# White lines represent missing data
msno.matrix(only_missing_data)

#### Heatmap

In [ ]:
# Positive correlation is proportional to the level of darkness in blue as indicated by the bar on the right side.

msno.heatmap(only_missing_data)

#### Dendogram

To interpret this graph, read it from a top-down perspective. Cluster leaves which linked together at a distance of zero fully predict one another's presence—one variable might always be empty when another is filled, or they might always both be filled or both empty, and so on. 

Cluster leaves which split close to zero, but not at it, predict one another very well, but still imperfectly. If your own interpretation of the dataset is that these columns actually are or ought to be match each other in nullity, then the height of the cluster leaf tells you, in absolute terms, how often the records are "mismatched", how many values you would have to fill in or drop, if you are so inclined.

In [ ]:
type(only_missing_data)
msno.dendrogram(only_missing_data)

#### Bar plot of missing values

It shows bars that are proportional to the number of non-missing values 
as well as providing the actual number of missingvalues. 
We get an idea of how much of each column is missing.


In [ ]:
# msno.bar(only_missing_data)
missing = training_data.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()  #   Height of bar shown number of rows with missing data

>_**Conclusions**_
> It is prudent to drop the PoolQC, MiscFeature, Alley and Fence features - there is simply too much data missing.  For the FireplaceQu feature, around 50% data is missing, but this seems to be the "rule of thumb" threshhold where it should be explored how much information is in the feature, and whether it is worth imputing values.  There is no objective value (that I could find), and what percentage missing data suggests dropping the whole feature depends on the situation.

Some resources with information on a rule of thumb to follow:

* [AnalyticsVidhya](https://discuss.analyticsvidhya.com/t/what-should-be-the-allowed-percentage-of-missing-values/2456): maybe 50%
* [Dan Berdikulov](https://medium.com/@danberdov/dealing-with-missing-data-8b71cd819501#:~:text=As%20a%20rule%20of%20thumb,the%20variable%20should%20be%20considered.): 60% - 70%


#### Drop columns
>_Make sure to drop in both training and competition sets_

In [ ]:
cd = competition_data.copy()

In [ ]:
print(type(X))
print(type(competition_data))
print(type(cd))

In [ ]:
X = X.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'],
           axis=1).copy()  #   Our training data
cd = cd.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'],
             axis=1).copy()  #   Our competition data

In [ ]:
X.info()
cd.info()

## Dealing with categorical data
>
>Internally, XGBoost models represent all problems as a regression predictive modeling problem that only takes numerical values as input. If your data is in a different form, it must be prepared into the expected format. See [Data Preparation for Gradient Boosting with XGBoost in Python](https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/)
>
>_**Remember to save the label encoder as a separate object so that we can transform both the training and later the test and validation datasets using the same encoding scheme.**_
>
> >xgboost only deals with numeric columns.

if you have a feature [a,b,b,c] which describes a categorical variable (i.e. no numeric relationship)

Using LabelEncoder you will simply have this:

array([0, 1, 1, 2])
Xgboost will wrongly interpret this feature as having a numeric relationship! This just maps each string ('a','b','c') to an integer, nothing more.

Proper way

Using OneHotEncoder you will eventually get to this:

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])
This is the proper representation of a categorical variable for xgboost or any other machine learning tool.

Pandas get_dummies is a nice tool for creating dummy variables (which is easier to use, in my opinion).

One suggestion of code:
>ONE_HOT_COLS = ["categorical_col1", "categorical_col2", "categorical_col3"]
print("Starting DF shape: %d, %d" % df.shape)


for col in ONE_HOT_COLS:
    s = df[col].unique()

    # Create a One Hot Dataframe with 1 row for each unique value
    one_hot_df = pd.get_dummies(s, prefix='%s_' % col)
    one_hot_df[col] = s

    print("Adding One Hot values for %s (the column has %d unique values)" % (col, len(s)))
    pre_len = len(df)

    # Merge the one hot columns
    df = df.merge(one_hot_df, on=[col], how="left")
    assert len(df) == pre_len
    print(df.shape)

### OneHotEncode Categorical Data
>
[Article on encoding categorical data](https://www.datacamp.com/community/tutorials/categorical-data) - used as inspiration for this code

In [64]:
categorical_features_indices = np.where(
    X.dtypes != np.float)[0]  # List of categorical features indices, int64
#  Off CatBoost training website

In [ ]:
categorical_features_indices.dtype

In [65]:
X = X.fillna(-999).copy()
cd = cd.fillna(-999).copy()

In [ ]:
X_play = X.iloc[:5, 0:2].copy()
X_play

In [ ]:
pd.get_dummies(X_play['MSSubClass'])

In [ ]:
pd.get_dummies(X_play['MSZoning'])

In [ ]:
pd.get_dummies(data=X_play, columns=['MSSubClass', 'MSZoning'])

In [ ]:
list = ['MSSubClass', 'MSZoning']
for element in list:
    print(type(element))

In [ ]:
pd.get_dummies(data=X_play, columns=list)

In [66]:
# From https://stackoverflow.com/questions/39923927/pandas-sklearn-one-hot-encoding-dataframe-or-numpy

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
columns_to_encode = categorical_features_indices
#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), X.columns[columns_to_encode].tolist())], remainder='passthrough')

In [67]:
# print(X.info(), "\n\n")
# print(type(columns_to_encode))
# columns_to_encode
# print("\n\n", X.columns[columns_to_encode].tolist())
list_to_encode = X.columns[columns_to_encode].tolist()
type(list_to_encode)
#list_to_encode

list

In [68]:
X_enc = pd.get_dummies(data=X, columns=list_to_encode).copy()
cd_enc = pd.get_dummies(data=X, columns=list_to_encode).copy()
type(X_enc.info())
type(cd_enc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 8574 entries, LotFrontage to SaleCondition_Partial
dtypes: float64(3), uint8(8571)
memory usage: 12.0 MB


NoneType

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 8574 entries, LotFrontage to SaleCondition_Partial
dtypes: float64(3), uint8(8571)
memory usage: 12.0 MB


NoneType

# Splitting the dataset into the Training set and Test set

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_enc,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=0)

<div class="alert alert-block alert-warning">  

_**Now everything is done up to X_train, X_test, y_train, y_test**_

We are now in familiar territory  
>
</div>

# Training XGBoost on the Training set - baseline model


In [ ]:
import xgboost as xgb

#########################################################################
#
# With gblinear the base model scores RMSE 57,877, R^2 0.51
# With gbtree the base model scores RMSE 39,527, R^2 0.77
#
#########################################################################

xg_reg = xgb.XGBRegressor(booster = 'gbtree', objective ='reg:squarederror', learning_rate = 0.3,
                n_estimators = 1000, eval_metric = 'rmse')

xg_reg.fit(X_train,y_train)

y_pred = xg_reg.predict(X_test)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print("RMSE: %f\n\n" % (rmse))
print("R^2: %f\n\n" % (r2))



In [ ]:
# Convert dataset to special XGBoost optimised data structure
dtrain = xgb.DMatrix(X_train, label=y_train, missing=-999.0)
dtest = xgb.DMatrix(X_test, label=y_test)
dcomp = xgb.DMatrix(cd_enc)

#   Specify parameters

evallist = [(dtest, 'eval'), (dtrain, 'train')]

#evals_result = {}

params = {
    'booster': 'gbtree', # gbtree beats gblinear - don't feel forced to fit linear to this dataset
    'objective': 'reg:squarederror',
    'learning_rate': 0.3,
#     'reg_alpha': 0,  #  L1 regularisation
#     'reg_lambda': 0,  # L2 regularisation
    'eval_metric': 'rmse',  # Used by Kaggle house price competition - this parameter is used for VALIDATION data
#    'evals': evallist,
    'verbose_eval': 1,  # If integer, value is printed every x boosting stages
    'early_stopping_rounds': 10,
    'verbosity': 1,  # 0 is silent, 3 is debug
}

#  Train the model
num_round = 1000
bst = xgb.train(params, dtrain, num_round, evallist)

# Evaluate how good the model is
y_pred = bst.predict(dtest)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print("RMSE: %f\n\n" % (rmse))
print("R^2: %f\n\n" % (r2))

In [80]:
type(params)

dict

# At this stage, the basic model works with the special DMatrix format and parametrisation

# Applying k-Fold Cross Validation to measure performance
>
> The result is a more reliable estimate of the performance of the algorithm on new data given your test data. It is more accurate because the algorithm is trained and evaluated multiple times on different data.
>
> We can use k-fold cross validation support provided in scikit-learn. First we must create the KFold object specifying the number of folds and the size of the dataset. We can then use this scheme with the specific dataset. The cross_val_score() function from scikit-learn allows us to evaluate a model using the cross validation scheme and returns a list of the scores for each model trained on each fold.
>
>
_**What about  XGBoost’s built-in Cross Validation ???**_


In [ ]:
# from sklearn.model_selection import cross_val_score
# accuracies = cross_val_score(estimator = xg_reg, X = X_train, y = y_train, cv = 10)
# print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
# import matplotlib.pyplot as plt

# xgb.plot_tree(xg_reg,num_trees=0)
# plt.rcParams['figure.figsize'] = [50, 10]
# plt.show()

# Exploring 


https://xgboost.readthedocs.io/en/latest/get_started.html

In [ ]:

#   Specify parameters

evallist = [(dtest, 'eval'), (dtrain, 'train')]

evals_result = {}

params = {
    'booster': 'gbtree', # gbtree beats gblinear - don't feel forced to fit linear to this dataset
    'objective': 'reg:squarederror',
    'learning_rate': 0.3,
#     'reg_alpha': 0,  #  L1 regularisation
#     'reg_lambda': 0,  # L2 regularisation
    'eval_metric': 'rmse',  # Used by Kaggle house price competition - this parameter is used for VALIDATION data
#    'evals': evallist,
    'verbose_eval': 1,  # If integer, value is printed every x boosting stages
    'early_stopping_rounds': 10,
    'verbosity': 1,  # 0 is silent, 3 is debug
}

#  Train the model
num_round = 1000
bst = xgb.train(params, dtrain, num_round, evallist)

# Evaluate how good the model is
y_pred = bst.predict(dtest)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print("RMSE: %f\n\n" % (rmse))
print("R^2: %f\n\n" % (r2))

In [ ]:
# # https://xgboost.readthedocs.io/en/latest/python/python_api.html
# # https://stackoverflow.com/questions/47152610/what-is-the-difference-between-xgb-train-and-xgb-xgbregressor-or-xgb-xgbclassif
# model = xgb.train(
#     params,
#     dtrain,
#     num_boost_round=num_rounds,
#     evals_result=evals_result
# )
# print(evals_result)
# y_pred = model.predict(dtest)
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print("RMSE: %f" % (rmse))

In [ ]:
#from sklearn.model_selection import cross_val_score

num_round = 10
#model = xgb.train(params, dtrain, num_round, evallist)

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_round,
    seed=42,
    nfold=5,
    metrics={'rmse'},
    early_stopping_rounds=10
)
cv_results

# result = xgb.cv(params=params, dtrain=dtrain, num_boost_round=num_round, early_stopping_rounds=10)
# print(result)
# accuracies = cross_val_score(estimator=model, X=X_enc, y=y_train, cv=10)
# print("Accuracy: {:.2f} %".format(accuracies.mean() * 100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

# Now start tuning
From https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f

In [ ]:
# Parameters max_depth and min_child_weight

gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]

# Define initial best params and RMSE
min_rmse = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    # Update best RMSE
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))

In [ ]:
# From previous round optimization
params['max_depth'] = 10
params['min_child_weight'] = 6

#  Parameters subsample and colsample_bytree
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

min_rmse = float("Inf")
best_params = None

# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (subsample,colsample)
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))

In [ ]:
# Update params from above
params['subsample'] = .7
params['colsample_bytree'] = 1.0

#  Now run model with these optimal values
num_round = 1000
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best RMSE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))


# Hyperopt
From here: https://towardsdatascience.com/an-example-of-hyperparameter-optimization-on-xgboost-lightgbm-and-catboost-using-hyperopt-12bc41a271e


fmin() is the main function in hyperopt for optimization. 
It accepts four basic arguments and output the optimized parameter set:
* Objective Function — fn
* Search Space — space
* Search Algorithm — algo
* (Maximum) no. of evaluations — max_evals

We may also pass a Trials object to the trials argument which keeps track of the whole process. In order to run with trails the output of the objective function has to be a dictionary including at least the keys 'loss' and 'status' which contain the result and the optimization status respectively. The interim values could be extracted by the following:
* trials.trials - a list of dictionaries contains all relevant information
* trials.results - a list of dictionaries collecting the function outputs
* trials.losses() - a list of losses (float for each 'ok' trial)
* trials.statuses() - a list of status strings
* trials.vals - a dictionary of sampled parameters



In [70]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

###############################################################
#
#   Need to compare the output with the "manual" result above - 36,106.71
#
###############################################################

# Choose hyperparameter domain to search over
space = {
    'booster': 'gbtree',
    'learning_rate': 0.3,
    'max_depth': hp.choice('max_depth', np.arange(5, 30, 5, dtype=int)),
    'n_estimators': hp.choice('n_estimators', np.arange(100, 1000, 200, dtype=int)),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.1, 1.5, 0.4),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 100, 50, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 0.9, 0.4),
    'eta': hp.quniform('eta', 0.05, 0.7, 0.2),
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
}

In [ ]:

# from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
# import matplotlib.pyplot as plt

# # Define objective function
# def f(x):
#     return {'loss': x ** 2 - x, 'status': STATUS_OK}

# # Run hyperopt optimization
# trials = Trials()
# result = fmin(
#     fn=f,                           # objective function
#     space=hp.uniform('x', -1, 1),   # parameter space
#     algo=tpe.suggest,               # surrogate algorithm
#     max_evals=500,                  # no. of evaluations
#     trials=trials                   # trials object that keeps track of the sample results (optional)
# )

# # Print the optimized parameters
# print(result)   # {'x': 0.5000833960783931}

# # Extract and plot the trials 
# x = trials.vals['x']
# y = [x['loss'] for x in trials.results]
# plt.scatter(x, y)

In [73]:
from hyperopt import hp
import numpy as np
from sklearn.metrics import mean_squared_error


# XGB parameters
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}
xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


# LightGBM parameters
lgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}
lgb_fit_params = {
    'eval_metric': 'l2',
    'early_stopping_rounds': 10,
    'verbose': False
}
lgb_para = dict()
lgb_para['reg_params'] = lgb_reg_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


# CatBoost parameters
ctb_reg_params = {
    'learning_rate':     hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
    'max_depth':         hp.choice('max_depth',         np.arange(5, 16, 1, dtype=int)),
    'colsample_bylevel': hp.choice('colsample_bylevel', np.arange(0.3, 0.8, 0.1)),
    'n_estimators':      100,
    'eval_metric':       'RMSE',
}
ctb_fit_params = {
    'early_stopping_rounds': 10,
    'verbose': False
}
ctb_para = dict()
ctb_para['reg_params'] = ctb_reg_params
ctb_para['fit_params'] = ctb_fit_params
ctb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

In [74]:
# https://towardsdatascience.com/an-example-of-hyperparameter-optimization-on-xgboost-lightgbm-and-catboost-using-hyperopt-12bc41a271e

import lightgbm as lgb
import xgboost as xgb
import catboost as ctb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials


class HPOpt(object):

    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test  = X_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def lgb_reg(self, para):
        reg = lgb.LGBMRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def ctb_reg(self, para):
        reg = ctb.CatBoostRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def train_reg(self, reg, para):
        reg.fit(self.X_train, self.y_train,
                eval_set=[(self.X_train, self.y_train), (self.X_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.X_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [75]:
obj = HPOpt(X_train, X_test, y_train, y_test)

xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)
lgb_opt = obj.process(fn_name='lgb_reg', space=lgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)
ctb_opt = obj.process(fn_name='ctb_reg', space=ctb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

#  See how good the models are:

# XGBoost
y_pred = xgb_opt.predict(y_test)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print("XGBoost\n")
print("RMSE: %f\n\n" % (rmse))
print("R^2: %f\n\n" % (r2))

# LightGBM
y_pred = lgb_opt.predict(y_test)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print("LGB\n")
print("RMSE: %f\n\n" % (rmse))
print("R^2: %f\n\n" % (r2))

# CatBoost
y_pred = ctb_opt.predict(y_test)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print("CatBoost\n")
print("RMSE: %f\n\n" % (rmse))
print("R^2: %f\n\n" % (r2))

100%|██████████| 100/100 [19:20<00:00, 11.60s/trial, best loss: 32335.596084526587]
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > 

AttributeError: 'tuple' object has no attribute 'predict'

In [84]:
type(xgb_opt)

xgb_opt[0]
# lgb_opt
# ctb_opt

tuple

{'colsample_bytree': 3,
 'learning_rate': 1,
 'max_depth': 4,
 'min_child_weight': 3,
 'subsample': 0.9376570860012425}

In [90]:

#######################################
# Now train model with the "ideal" parameters
#####################################


#   Specify parameters

evallist = [(dtest, 'eval'), (dtrain, 'train')]

evals_result = {}

params = {
    'booster': 'gbtree', # gbtree beats gblinear - don't feel forced to fit linear to this dataset
    'objective': 'reg:squarederror',
#    'colsample_bytree': 3,
    'max_depth': 4,
    'min_child_weight': 3,
    'subsample': 0.9376570860012425,
    'learning_rate': 1,
#     'reg_alpha': 0,  #  L1 regularisation
#     'reg_lambda': 0,  # L2 regularisation
    'eval_metric': 'rmse',  # Used by Kaggle house price competition - this parameter is used for VALIDATION data
#    'evals': evallist,
    'verbose_eval': 1,  # If integer, value is printed every x boosting stages
    'early_stopping_rounds': 10,
    'verbosity': 1,  # 0 is silent, 3 is debug
}

#  Train the model
num_round = 1000
bst = xgb.train(params, dtrain, num_round, evallist)

# Evaluate how good the model is
y_pred = bst.predict(dtest)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print("RMSE: %f\n\n" % (rmse))
print("R^2: %f\n\n" % (r2))

ain-rmse:62.94508
[581]	eval-rmse:43171.11719	train-rmse:62.02114
[582]	eval-rmse:43171.52734	train-rmse:61.60399
[583]	eval-rmse:43171.05078	train-rmse:61.61120
[584]	eval-rmse:43171.35156	train-rmse:61.55329
[585]	eval-rmse:43171.42188	train-rmse:61.08472
[586]	eval-rmse:43171.30859	train-rmse:60.80474
[587]	eval-rmse:43171.66016	train-rmse:60.80100
[588]	eval-rmse:43172.24219	train-rmse:59.85804
[589]	eval-rmse:43170.60938	train-rmse:59.30514
[590]	eval-rmse:43170.81641	train-rmse:58.95732
[591]	eval-rmse:43170.30859	train-rmse:58.48189
[592]	eval-rmse:43170.25000	train-rmse:58.27849
[593]	eval-rmse:43170.47656	train-rmse:58.02854
[594]	eval-rmse:43170.23047	train-rmse:57.24585
[595]	eval-rmse:43168.30859	train-rmse:56.96907
[596]	eval-rmse:43168.37891	train-rmse:56.47749
[597]	eval-rmse:43168.40625	train-rmse:56.56363
[598]	eval-rmse:43168.53516	train-rmse:55.96230
[599]	eval-rmse:43168.77344	train-rmse:55.46189
[600]	eval-rmse:43168.36328	train-rmse:54.58975
[601]	eval-rmse:43167.

In [ ]:
# Define objective function for hyperopt

def objective(params):
    model = xgb(**params)
    num_round=10
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=num_round,
        evals=[(dtest, "Test")],
        early_stopping_rounds=10
        )
    # model = XGBRegressor(**params)

    # model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)],
    #           verbose=False, early_stopping_rounds=10)

    # y_pred = model.predict(X_test)
    # score = math.sqrt(mean_squared_error(y_test, y_pred))
    # print('RMSE: {:0.2f}'.format(score))
    # return {'loss': score, 'status': STATUS_OK}


    # num_round = 1000

    


In [ ]:
# Now run the actual pyperopt optimisation

trials = Trials()
best_params = fmin(
    fn=objective, 
    space=space, 
    algo=tpe.suggest, 
    max_evals=2, #  Tune this: maximum no of evaluations
    trials=trials #  trials object that keeps track of the sample results
    )

print(best_params)

# Extract and plot the trials 
x = trials.vals['x']
y = [x['loss'] for x in trials.results]
plt.scatter(x, y)


In [ ]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'reg:squarederror'}
param['nthread'] = 4
param[
    'eval_metric'] = 'rmse'  #  This is what the Kaggel house price competition uses to score

#  Instantiate an XGBoost regressor object
xg_reg = xgb(booster='gblinear',
             objective='reg:squarederror',
             colsample_bytree=0.3,
             learning_rate=0.3,
             max_depth=10,
             alpha=0,
             n_estimators=1000,
             eval_metric='rmse')

# num_round = 15
# initial_trees = xgb.train(param, dtrain, num_round)

xg_reg.fit(X_train, y_train)

# dtest = xgb.DMatrix(X_test)
# y_pred = bst.predict(dtest)

y_pred = xg_reg.predict(X_test)

math.sqrt(mean_squared_error(y_test, y_pred))

#model = XGBRegressor(space)

In [ ]:
model.fit(X_train,
          y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False,
          early_stopping_rounds=10)

y_pred = model.predict(X_test)
score = math.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE: {:0.2f}'.format(score))

In [ ]:
from hyperopt.plotting import main_plot_history
main_plot_history(trials)

In [ ]:
model

In [ ]:
y_pred = model.predict(X_test)
print("R^2\n\n")
r2_score(y_test, y_pred)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=model, X=X_train, y=y_train, cv=10)
print("Accuracy: {:.2f} %".format(accuracies.mean() * 100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(model, num_trees=0)
plt.rcParams['figure.figsize'] = [500, 200]
plt.show()

# Another optimisation effort
From https://towardsdatascience.com/how-to-get-started-on-kaggle-competitions-68b91e3e803a
and https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost

In [ ]:
# parameters to be searched over
param_grid = {
    'model__n_estimators': [10, 50, 100, 200, 400, 600],
    'model__max_depth': [2, 3, 5, 7, 10],
    'model__min_child_weight': [0.0001, 0.001, 0.01],
    'model__learning_rate': [0.01, 0.1, 0.5, 1]
}

xg_reg = xgb.XGBRegressor(booster='gblinear',
                          objective='reg:squarederror',
                          colsample_bytree=0.3,
                          learning_rate=0.3,
                          max_depth=10,
                          alpha=0,
                          n_estimators=1000,
                          eval_metric='rmse')

# find the best parameter
kfold = KFold(shuffle=True, random_state=0)
grid_search = GridSearchCV(xg_reg,
                           param_grid,
                           scoring='neg_root_mean_squared_error',
                           cv=kfold,
                           n_jobs=-1)
grid_result = grid_search.fit(X_train, y_train)

# folds = 3
# param_comb = 5

# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3, random_state=1001 )

# # Here we go
# start_time = timer(None) # timing starts from this point for "start_time" variable
# random_search.fit(X, Y)
# timer(start_time) # timing ends here for "start_time" variable

#  Print gridsearch results and save to file
print('\n All results:')
print(grid_search.cv_results_)
print('\n Best estimator:')
print(grid_search.best_estimator_)
print(
    '\n Best normalized gini score for %d-fold search with %d parameter combinations:'
    % (folds, param_comb))
print(grid_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(grid_search.best_params_)
results = pd.DataFrame(grid_search.cv_results_)
results.to_csv('xgb-gridsearchcv-results-01.csv', index=False)

In [ ]:
#  Need to figure out how to extract the optimised model, then run the cells below to test how good it is
######  Looks like it is in "best estimator"

In [ ]:
y_pred = model.predict(X_test)
print("R^2\n\n")
r2_score(y_test, y_pred)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=model, X=X_train, y=y_train, cv=10)
print("Accuracy: {:.2f} %".format(accuracies.mean() * 100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(model, num_trees=0)
plt.rcParams['figure.figsize'] = [500, 200]
plt.show()